In [1]:
# %load_ext autoreload
# %autoreload 2
%matplotlib inline

In [2]:
from branchynet.net import BranchyNet
from branchynet.links import *
import chainer.functions as F
import chainer.links as L
from branchynet import utils, visualize
from chainer import cuda

Define Network

In [3]:
from networks import resnet_cifar10

branchyNet = resnet_cifar10.get_network()
if cuda.available:
    branchyNet.to_gpu()
branchyNet.training()

Import Data

In [10]:
from datasets import pcifar10

x_train, y_train, x_test, y_test = pcifar10.get_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
training_size=100
x_train, y_train, x_test, y_test = x_train[:training_size*6], y_train[:training_size*6], x_test[:training_size], y_test[:training_size]
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

#training size = 500, each epoch we need at least 2 minutes.
#this app need 30 cores and at least 20GB memory for UTSA server when training size is 500 .

((50000, 3, 32, 32), (50000,), (10000, 3, 32, 32), (10000,))
((600, 3, 32, 32), (600,), (100, 3, 32, 32), (100,))


Settings

In [11]:
TRAIN_BATCHSIZE = 512
TEST_BATCHSIZE = 1
TRAIN_NUM_EPOCHS = 50

Train Main Network

In [ ]:
main_loss, main_acc, main_time = utils.train(branchyNet, x_train, y_train, main=True, batchsize=TRAIN_BATCHSIZE,
                                             num_epoch=TRAIN_NUM_EPOCHS)

1620772176.4
[ 95  42 157 390 291  41 444 166 179 420 173 152 451 161 307 136  21 401
 406 490 245  58 564  93 447 153 121 482 428 120 566 268 233 599 478 208
  65 195 202 203 322 422  13 539 530 568 597  73 147 164 105 210  36 312
 135   2 272 101 395 575 293  18 503 109 411 114  23 339  89 143 187 549
 104 188 377 523 225 330 288  49 418 536  48 592 125 496 306 477 354 459
  61 235 223 224 113 577 495 298 446  39 511 219 148 280 382   5 594 463
 230 424 481 129 167 441  69 456 176 253 378 269 256 400 242 550 487 543
 480 134 149  96 207 177  24 309 254 467 485 562 468 578 174 357 526 538
 217  29 251 559 342 458 414 416   3 227 368 442  53 529 119 353 409 457
 131 367 365 376 370 397  76 586 289 282 519 103 525   6 517 151 252  27
 589  77 527 281 165 193 385 345 473 155 160 580 356  99  87 133 209 548
  92 123 501 335 383 427 228 182 321 531 425 279 453 563 199 423 260  88
 433 100 436 204 569 212 386 582 380 560 352 494  78 311 389 249 591 264
  51 261 185 117 590 497 130 226 229 2

Train BranchyNet

In [ ]:
TRAIN_NUM_EPOCHS = 50
branch_loss, branch_acc, branch_time = utils.train(branchyNet, x_train, y_train, batchsize=TRAIN_BATCHSIZE,
                                                   num_epoch=TRAIN_NUM_EPOCHS)

#set network to inference mode
branchyNet.testing()

Visualizing Network Training

In [ ]:
visualize.plot_layers(main_loss, xlabel='Epochs', ylabel='Training Loss')
visualize.plot_layers(main_acc, xlabel='Epochs', ylabel='Training Accuracy')

In [ ]:
visualize.plot_layers(zip(*branch_loss), xlabel='Epochs', ylabel='Training Loss')
visualize.plot_layers(zip(*branch_acc), xlabel='Epochs', ylabel='Training Accuracy')

Run test suite and visualize

In [ ]:
#set network to inference mode
branchyNet.testing()
branchyNet.verbose = False
if cuda.available:
    branchyNet.to_gpu()
g_baseacc, g_basediff, _, _ = utils.test(branchyNet,x_test,y_test,main=True,batchsize=TEST_BATCHSIZE)
g_basediff = (g_basediff / float(len(y_test))) * 1000.

#branchyNet.to_cpu()
c_baseacc, c_basediff, _, _ = utils.test(branchyNet,x_test,y_test,main=True,batchsize=TEST_BATCHSIZE)
c_basediff = (c_basediff / float(len(y_test))) * 1000.

In [ ]:
# Specify thresholds
thresholds = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1., 2., 3., 5., 10.]

In [ ]:
#GPU
if cuda.available:
    branchyNet.to_gpu()
g_ts, g_accs, g_diffs, g_exits = utils.screen_branchy(branchyNet, x_test, y_test, thresholds,
                                                    batchsize=TEST_BATCHSIZE, verbose=True)


#convert to ms
g_diffs *= 1000.

In [ ]:
visualize.plot_line_tradeoff(g_accs, g_diffs, g_ts, g_exits, g_baseacc, g_basediff, all_samples=False, inc_amt=-0.0001000,
                             our_label='BranchyResNet', orig_label='ResNet', xlabel='Runtime (ms)', 
                             title='ResNet GPU', output_path='_figs/resnet_gpu.pdf')

In [ ]:
#CPU
branchyNet.to_cpu()
c_ts, c_accs, c_diffs, c_exits  = utils.screen_branchy(branchyNet, x_test, y_test, thresholds,
                                                     batchsize=TEST_BATCHSIZE, verbose=True)

#convert to ms
c_diffs *= 1000.

In [ ]:
visualize.plot_line_tradeoff(c_accs, c_diffs, c_ts, c_exits, c_baseacc, c_basediff, all_samples=False, inc_amt=-0.0001000,
                             our_label='BranchyResNet', orig_label='ResNet', xlabel='Runtime (ms)',
                             title='ResNet CPU', output_path='_figs/resnet_cpu.pdf')

In [ ]:
print 'CPU Results:'
utils.branchy_table_results('ResNet', c_baseacc, c_basediff, c_accs, c_diffs, c_exits, c_ts)

print '\n\n'

print 'GPU Results:'
utils.branchy_table_results('ResNet', g_baseacc, g_basediff, g_accs, c_diffs, g_exits, g_ts)

Save model/data

In [ ]:
import dill
branchyNet.to_cpu()
with open("_models/resnet_cifar10.bn", "wb") as f:
    dill.dump(branchyNet, f)
with open("_models/resnet_cifar10_gpu_results.pkl", "wb") as f:
    dill.dump({'accs': g_accs, 'rt': g_diffs, 'exits': g_exits, 'ts': g_ts, 'baseacc': g_baseacc, 'basediff': g_basediff}, f)
with open("_models/resnet_cifar10_cpu_results.pkl", "wb") as f:
    dill.dump({'accs': c_accs, 'rt': c_diffs, 'exits': c_exits, 'ts': c_ts, 'baseacc': c_baseacc, 'basediff': c_basediff}, f)